# OCI Language Service Sample Example Call through AIDP notebook


####  This sample shows how to call **OCI AI Language** Translation (REST) from Python with a **user principal** (API key) and `oci.signer.Signer`.

## Prerequisites

#### Install oci package on the compute cluster where this notebook will execute. Eg. Add oci in the requirements.txt. Please refer https://docs.oracle.com/en/cloud/paas/ai-data-platform/aidug/manage-libraries.html for more details
####  Add IAM policy to add allow calling OCI Langurage Service. Please refer https://docs.oracle.com/en-us/iaas/Content/language/using/policies.htm for more details

#### Configure auth via API key
#### Fill the placeholders below. The private key must match the fingerprint on your user.

In [1]:
import oci
import requests
import json
from oci.signer import Signer

# === PLACEHOLDERS: fill these ===
USER_OCID      = "<YOUR_USER_OCID>"
TENANCY_OCID   = "<YOUR_TENANCY_OCID>"
FINGERPRINT    = "<YOUR_API_KEY_FINGERPRINT>"
REGION         = "<YOUR_OCI_REGION>"            # e.g., "uk-london-1", "us-ashburn-1"
PRIVATE_KEY_PEM_PATH = "<PATH_TO_PRIVATE_KEY_PEM>"  # e.g., "/Workspace/keys/oci_api_key.pem"


# Build a reusable request signer (user principal)
signer = Signer(
    tenancy=TENANCY_OCID,
    user=USER_OCID,
    fingerprint=FINGERPRINT,
    private_key_file_location=PRIVATE_KEY_PEM_PATH, 
)

print("Signer ready for region:", REGION)

Signer ready for region: uk-london-1


#### Prepare endpoint and payload (EN → ES)
#### Endpoint pattern: https://language.aiservice.{region}.oci.oraclecloud.com/20221001/actions/batchLanguageTranslation

In [1]:
endpoint = f"https://language.aiservice.{REGION}.oci.oraclecloud.com/20221001/actions/batchLanguageTranslation"
headers  = {"Content-Type": "application/json"}

# Example batch (English → Spanish)
body = {
    "documents": [
        {"key": "1", "text": "Hello, how can I help you today?", "languageCode": "en"},
        {"key": "2", "text": "We are integrating OCI services with AI Data Platform.", "languageCode": "en"},
        {"key": "3", "text": "Thank you for your patience and support.", "languageCode": "en"},
    ],
    "targetLanguageCode": "es"
}

print(json.dumps(body_en_to_es, indent=2))

{
  "documents": [
    {
      "key": "1",
      "text": "Hello, how can I help you today?",
      "languageCode": "en"
    },
    {
      "key": "2",
      "text": "We are integrating OCI services with AI Data Platform.",
      "languageCode": "en"
    },
    {
      "key": "3",
      "text": "Thank you for your patience and support.",
      "languageCode": "en"
    }
  ],
  "targetLanguageCode": "es"
}


#### Call Language (EN → ES), then back-translate (ES → EN)


In [1]:
# Step 4: Make the POST request
response = requests.post(endpoint, auth=signer, headers=headers, data=json.dumps(body))

# Step 5: Show result
if response.status_code == 200:
    print("OCI Language Service called successfully!")
    data = response.json()
    es_docs = data.get("documents", [])
    es_texts = [doc["translatedText"] for doc in es_docs]

    # --- Step 4: ES -> EN (back translation) ---
    back_body = {
        "documents": [{"key": str(i+1), "text": t, "languageCode": "es"} for i, t in enumerate(es_texts)],
        "targetLanguageCode": "en"
    }

    back_resp = requests.post(endpoint, auth=signer, headers=headers, data=json.dumps(back_body))
    if back_resp.status_code == 200:
        back_data = back_resp.json()
        back_texts = [doc["translatedText"] for doc in back_data.get("documents", [])]

        # --- Print side by side ---
        for i, (orig, es, back_en) in enumerate(zip([d["text"] for d in body["documents"]], es_texts, back_texts), start=1):
            print(f"\nSentence {i}:")
            print(" Original EN :", orig)
            print(" Spanish     :", es)
            print(" Back to EN  :", back_en)
    else:
        print("Error in back translation:", back_resp.status_code, back_resp.text)

else:
    print("Error calling OCI Language Service")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.text}")

OCI Language Service called successfully!

Sentence 1:
 Original EN : Hello, how can I help you today?
 Spanish     : Hola, ¿cómo puedo ayudarte hoy?
 Back to EN  : Hi, how can I help you today?

Sentence 2:
 Original EN : We are integrating OCI services with AI Data Platform.
 Spanish     : Estamos integrando los servicios de OCI con AI Data Platform.
 Back to EN  : We are integrating OCI services with AI Data Platform.

Sentence 3:
 Original EN : Thank you for your patience and support.
 Spanish     : Gracias por su paciencia y apoyo.
 Back to EN  : Thank you for your patience and support.
